# Entraînement et Soumission Kaggle - Modèle Multimodal

Ce notebook regroupe l'ensemble du pipeline pour entraîner un modèle de classification de tweets (multimodal : texte + métadonnées) et générer un fichier de soumission pour Kaggle.

## 1. Importations et Configuration de l'Environnement

In [1]:
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import pickle
import random
import os
import pprint

# Configuration du device
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# Fixation de la graine aléatoire pour la reproductibilité
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

train_path="/kaggle/input/influencers-or-observers-predicting-social-roles/Kaggle2025/train.jsonl"
test_path='/kaggle/input/influencers-or-observers-predicting-social-roles/Kaggle2025/kaggle_test.jsonl'

print(os.path.exists(train_path) and os.path.exists(test_path))


Using device: cuda
True


## 2. Configuration des Hyperparamètres

Nous définissons ici une classe de configuration pour centraliser tous les hyperparamètres du modèle et de l'entraînement.

In [ ]:
class TrainingConfig:
    # Modèle
    transformer_name = '/kaggle/input/twitterroberta/transformers/default/1/twitter-xlm-roberta-base'

    metadata_dim: int = 8
    meta_hidden_dim: int = 32
    text_hidden_dim: int = 768
    fusion_hidden_dim: int = 256
    
    # Entraînement
    max_length: int = 220 #vérifier quelle doit être la bonne longueur mtn que j'ajoute la description et le nom du site
    batch_size: int = 16
    num_epochs: int = 4
    lr_transformer: float = 2e-5
    lr_head: float = 1e-3
    weight_decay: float = 0.01
    seed: int = 42
    freeze_transformer: bool = False

cfg = TrainingConfig()
print("Configuration actuelle :")
pprint.pprint(cfg.__dict__)

Configuration actuelle :
{}


## 3. Chargement et Prétraitement des Données

Cette section contient les fonctions pour charger les fichiers JSONL, extraire les caractéristiques textuelles et les métadonnées, et normaliser les données.

In [ ]:
def extract_features_from_row(row):
    """
    Extrait le texte complet et les métadonnées d'une ligne de données (tweet).
    """
    # 1. Extraction du texte complet
    text = row.get('text', '')
    # Gestion des tweets étendus (si json_normalize a aplati la structure)
    if pd.notna(row.get('extended_tweet.full_text')):
        text = row['extended_tweet.full_text']
        
    # Normalisation du texte et ajout de l'URL utilisateur et de sa bio
    text = str(text).replace('\n', ' ').strip()
    user_url = row.get('user.url', '')
    user_description = row.get('user.description', '')
    quoted_status_full_text = row.get('quoted_status.full_text', '')
    text = f"[TEXTE] {text}"
    if pd.notna(user_description) and user_description:
        text = f"{text} [PERSONAL_BIO] {user_description}"
    if pd.notna(user_url) and user_url:
        text = f"{text} [PERSONAL_URL] {user_url}"
    #if pd.notna(quoted_status_full_text) and quoted_status_full_text:
    #    text = f"{text} [QUOTED_STATUS] {quoted_status_full_text}"
    #unquote later mais je veux tester le nombre de features d'abord
    

        
    # 2. Extraction des métadonnées
    def safe_get(key, default=0):
        val = row.get(key, default)
        if pd.isna(val):
            return default
        return val

    metadata = {
        'user_default_profile': int(safe_get('user.default_profile', False)),
        'user_profile_use_background_image': int(safe_get('user.profile_use_background_image', False)),
        'user_statuses_count': float(safe_get('user.statuses_count', 0)),
        'user_profile_background_tile': int(safe_get('user.profile_background_tile', False)),
        'user_geo_enabled': int(safe_get('user.geo_enabled', False)),
        'user_is_translator': int(safe_get('user.is_translator', False)),
        'user_favourites_count': float(safe_get('user.favourites_count', 0)),
        'user_listed_count': float(safe_get('user.listed_count', 0)),
        'quoted_is_quote_status': int(safe_get('quoted_status.is_quote_status', False)),
        'quoted_favorite_count': float(safe_get('quoted_status.favorite_count', 0)),
        'quoted_quote_count': float(safe_get('quoted_status.quote_count', 0)),
        'quoted_possibly_sensitive': int(safe_get('quoted_status.possibly_sensitive', False)),
        'quoted_user_listed_count': float(safe_get('quoted_status.user.listed_count', 0)),
    }
    
    return pd.Series({'full_text': text, **metadata})

def load_data():
    # Chargement Train
    print(f"Chargement des données d'entraînement depuis {train_path}...")
    train_data_raw = []
    if os.path.exists(train_path):
        with open(train_path, 'r') as f:
            for line in f:
                train_data_raw.append(json.loads(line))
        df_train = pd.json_normalize(train_data_raw)
    else:
        print(f"Attention: {train_path} introuvable.")
        df_train = pd.DataFrame()
    
    # Chargement Test (Kaggle)
    print(f"Chargement des données de test depuis {test_path}...")
    test_data_raw = []
    if os.path.exists(test_path):
        with open(test_path, 'r') as f:
            for line in f:
                test_data_raw.append(json.loads(line))
        df_test = pd.json_normalize(test_data_raw)
    else:
        print(f"Attention: {test_path} introuvable.")
        df_test = pd.DataFrame()
    
    return df_train, df_test

def preprocess_data(df_train, df_test):
    # Extraction des features
    print("Extraction des features pour le train...")
    train_features = df_train.apply(extract_features_from_row, axis=1)
    
    print("Extraction des features pour le test...")
    test_features = df_test.apply(extract_features_from_row, axis=1)
    
    # Transformation Logarithmique pour certaines colonnes numériques
    log_cols = [
        'user_statuses_count', 
        'user_favourites_count', 
        'user_listed_count',
        'quoted_favorite_count',
        'quoted_quote_count',
        'quoted_user_listed_count'
    ]
    for col in log_cols:
        train_features[col] = np.log1p(train_features[col])
        test_features[col] = np.log1p(test_features[col])
        
    # Séparation des métadonnées pour le scaling
    metadata_cols = [c for c in train_features.columns if c != 'full_text']
    
    # Scaling (StandardScaler)
    print("Mise à l'échelle des métadonnées...")
    scaler = StandardScaler()
    # Fit sur le train uniquement
    metadata_train = scaler.fit_transform(train_features[metadata_cols].values)
    # Transform sur le test
    metadata_test = scaler.transform(test_features[metadata_cols].values)
    
    # Préparation des listes finales de dictionnaires
    train_list = []
    for i in range(len(train_features)):
        train_list.append({
            "text": train_features.iloc[i]['full_text'],
            "metadata": metadata_train[i],
            "label": df_train.iloc[i]['label']
        })
        
    test_list = []
    for i in range(len(test_features)):
        test_list.append({
            "text": test_features.iloc[i]['full_text'],
            "metadata": metadata_test[i],
            "challenge_id": df_test.iloc[i]['challenge_id']
        })
        
    return train_list, test_list, scaler

# Exécution du chargement (si les fichiers sont présents)
print(train_path)
if os.path.exists(train_path) and os.path.exists(test_path):
    df_train, df_test = load_data()
    train_data_processed, kaggle_data_processed, scaler = preprocess_data(df_train, df_test)
    print(f"Données traitées : {len(train_data_processed)} exemples d'entraînement, {len(kaggle_data_processed)} exemples de test.")
else:
    print("Fichiers de données non trouvés dans le répertoire")

/kaggle/input/influencers-or-observers-predicting-social-roles/Kaggle2025/train.jsonl
Chargement des données d'entraînement depuis /kaggle/input/influencers-or-observers-predicting-social-roles/Kaggle2025/train.jsonl...
Chargement des données de test depuis /kaggle/input/influencers-or-observers-predicting-social-roles/Kaggle2025/kaggle_test.jsonl...
Extraction des features pour le train...
Extraction des features pour le test...
Mise à l'échelle des métadonnées...
Données traitées : 154914 exemples d'entraînement, 103380 exemples de test.


## 4. Classe Dataset PyTorch

Définition de la classe `TweetDataset` pour gérer les données textuelles et numériques.

In [4]:
class TweetDataset(Dataset):
    def __init__(self, data, tokenizer, max_length, with_labels=True):
        """
        Args:
            data (list of dict): Liste d'échantillons.
            tokenizer: Tokenizer HuggingFace.
            max_length (int): Longueur maximale de la séquence.
            with_labels (bool): Si True, retourne aussi les labels.
        """
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.with_labels = with_labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item["text"]
        metadata = item["metadata"]

        # Tokenisation
        encoded = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )

        # Suppression de la dimension de batch ajoutée par le tokenizer (1, seq_len) -> (seq_len)
        input_ids = encoded["input_ids"].squeeze(0)
        attention_mask = encoded["attention_mask"].squeeze(0)

        # Conversion des métadonnées en tenseur float
        metadata_tensor = torch.tensor(metadata, dtype=torch.float32)

        result = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "metadata": metadata_tensor
        }

        if self.with_labels:
            label = item["label"]
            result["labels"] = torch.tensor(label, dtype=torch.long)

        return result

## 5. Architecture du Modèle Multimodal

Le modèle combine un Transformer (pour le texte) et un MLP (pour les métadonnées).

In [5]:
class MultimodalTweetClassifier(nn.Module):
    def __init__(self, 
                 transformer_name="cardiffnlp/twitter-xlm-roberta-base",
                 metadata_dim=8,
                 text_hidden_dim=768,
                 meta_hidden_dim=32,
                 fusion_hidden_dim=256):
        super(MultimodalTweetClassifier, self).__init__()
        
        # 1. Encodeur Transformer
        self.transformer = AutoModel.from_pretrained(transformer_name)
        
        # 2. MLP pour les métadonnées
        self.meta_mlp = nn.Sequential(
            nn.Linear(metadata_dim, meta_hidden_dim),
            nn.ReLU(),
            nn.LayerNorm(meta_hidden_dim)
        )
        
        # 3. Classifieur de fusion
        self.classifier = nn.Sequential(
            nn.Linear(text_hidden_dim + meta_hidden_dim, fusion_hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(fusion_hidden_dim, 2)  # 2 classes
        )

    def forward(self, input_ids, attention_mask, metadata):
        # Passage du texte dans le transformer
        outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        
        # Récupération du token CLS (premier token)
        cls_embedding = outputs.last_hidden_state[:, 0, :]
        
        # Passage des métadonnées dans le MLP
        meta_repr = self.meta_mlp(metadata)
        
        # Concaténation
        fused = torch.cat([cls_embedding, meta_repr], dim=1)
        
        # Classification finale
        logits = self.classifier(fused)
        
        return logits

## 6. Fonctions d'Entraînement et de Validation

Fonctions utilitaires pour exécuter une époque d'entraînement et évaluer le modèle.

In [6]:
def train_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    
    for batch in tqdm(loader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        metadata = batch['metadata'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        logits = model(input_ids, attention_mask, metadata)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    return total_loss / len(loader)

def eval_model(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Validation"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            metadata = batch['metadata'].to(device)
            labels = batch['labels'].to(device)
            
            logits = model(input_ids, attention_mask, metadata)
            loss = criterion(logits, labels)
            total_loss += loss.item()
            
            preds = torch.argmax(logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            
    return total_loss / len(loader), correct / total

## 7. Pipeline Principal d'Entraînement

Lancement de l'entraînement avec séparation train/validation, initialisation du modèle et boucle d'époques.

In [7]:
# Vérification que les données sont chargées
if 'train_data_processed' in locals():
    # Séparation Train / Validation
    train_list, val_list = train_test_split(
        train_data_processed, 
        test_size=0.2, 
        random_state=cfg.seed, 
        stratify=[x['label'] for x in train_data_processed]
    )
    
    print(f"Train size: {len(train_list)}, Val size: {len(val_list)}")

    # Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(cfg.transformer_name)

    # Datasets & DataLoaders
    train_dataset = TweetDataset(train_list, tokenizer, cfg.max_length, with_labels=True)
    val_dataset = TweetDataset(val_list, tokenizer, cfg.max_length, with_labels=True)

    train_loader = DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=cfg.batch_size, shuffle=False)

    # Initialisation du Modèle
    model = MultimodalTweetClassifier(
        transformer_name=cfg.transformer_name,
        metadata_dim=cfg.metadata_dim,
        text_hidden_dim=cfg.text_hidden_dim,
        meta_hidden_dim=cfg.meta_hidden_dim,
        fusion_hidden_dim=cfg.fusion_hidden_dim
    )
    model.to(device)
    
    # Optionnel : Geler le transformer
    if cfg.freeze_transformer:
        print("Gel des paramètres du Transformer...")
        for p in model.transformer.parameters():
            p.requires_grad = False

    # Optimiseur
    # On applique des learning rates différents pour le transformer et la tête de classification
    transformer_params = [p for p in model.transformer.parameters() if p.requires_grad]
    param_groups = []
    if len(transformer_params) > 0:
        param_groups.append({'params': transformer_params, 'lr': cfg.lr_transformer})
    param_groups.append({'params': model.meta_mlp.parameters(), 'lr': cfg.lr_head})
    param_groups.append({'params': model.classifier.parameters(), 'lr': cfg.lr_head})

    optimizer = AdamW(param_groups, weight_decay=cfg.weight_decay)
    criterion = nn.CrossEntropyLoss()

    # Boucle d'Entraînement
    best_val_acc = 0.0
    
    print("Début de l'entraînement...")
    for epoch in range(cfg.num_epochs):
        print(f"\nEpoch {epoch+1}/{cfg.num_epochs}")
        
        train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
        val_loss, val_acc = eval_model(model, val_loader, criterion, device)
        
        print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_multimodal_model.pt")
            print("--> Nouveau meilleur modèle sauvegardé.")
else:
    print("Données non chargées, impossible de lancer l'entraînement.")

Train size: 123931, Val size: 30983


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

Train Loss: 0.4143 | Val Loss: 0.4162 | Val Acc: 0.8125

Epoch 3/4


Validation: 100%|██████████| 1937/1937 [02:36<00:00, 12.39it/s]


Train Loss: 0.4208 | Val Loss: 0.4132 | Val Acc: 0.8149

Epoch 4/4


Validation: 100%|██████████| 1937/1937 [02:37<00:00, 12.31it/s]

Train Loss: 0.4205 | Val Loss: 0.4126 | Val Acc: 0.8164


## 8. Inférence et Génération du Fichier de Soumission

Chargement du meilleur modèle et prédiction sur le jeu de test Kaggle.

In [8]:
if 'kaggle_data_processed' in locals() and os.path.exists("best_multimodal_model.pt"):
    # Chargement du meilleur modèle
    print("Chargement du meilleur modèle pour l'inférence...")
    model.load_state_dict(torch.load("best_multimodal_model.pt", map_location=device))
    model.eval()

    # Dataset Kaggle
    kaggle_dataset = TweetDataset(kaggle_data_processed, tokenizer, cfg.max_length, with_labels=False)
    kaggle_loader = DataLoader(kaggle_dataset, batch_size=cfg.batch_size, shuffle=False)

    all_preds = []
    all_ids = []

    print("Lancement des prédictions sur le jeu de test Kaggle...")
    with torch.no_grad():
        batch_start_idx = 0
        for batch in tqdm(kaggle_loader, desc="Predicting"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            metadata = batch['metadata'].to(device)
            
            logits = model(input_ids, attention_mask, metadata)
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            
            all_preds.extend(preds)
            
            # Récupération des IDs correspondants
            batch_size = input_ids.size(0)
            batch_ids = [item['challenge_id'] for item in kaggle_data_processed[batch_start_idx : batch_start_idx + batch_size]]
            all_ids.extend(batch_ids)
            
            batch_start_idx += batch_size

    # Création du DataFrame de soumission
    results_df = pd.DataFrame({
        "ID": all_ids,
        "Prediction": all_preds
    })

    output_file = "submission.csv"
    results_df.to_csv(output_file, index=False)
    print(f"Prédictions sauvegardées dans {output_file}")
    print(results_df.head())
else:
    print("Impossible de lancer l'inférence (données manquantes ou modèle non entraîné).")

Chargement du meilleur modèle pour l'inférence...
Lancement des prédictions sur le jeu de test Kaggle...


Predicting: 100%|██████████| 6462/6462 [08:41<00:00, 12.40it/s]


Prédictions sauvegardées dans submission.csv
   ID  Prediction
0   0           1
1   2           1
2   4           0
3   8           1
4   9           0
